In [10]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tqdm import tqdm

In [12]:
import datetime

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
class VGGNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        
        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu",
            input_shape=input_shape
        ))

        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Flatten()),

        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(1000, activation="softmax"))

        self.compile(
            optimizer=Adam(0.001),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

In [16]:
num_epochs = 1
batch_size = 2
img_height = 224
img_width = 224
Input_shape = (img_width, img_height, 3)
num_classes = 3


model = VGGNet(Input_shape, num_classes=num_classes)

model.summary()

2024-07-28 14:08:32.759113: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 392.00MiB (rounded to 411041792)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-07-28 14:08:32.759221: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-07-28 14:08:32.759254: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 41, Chunks in use: 41. 10.2KiB allocated for chunks. 10.2KiB in use in bin. 2.7KiB client-requested in use in bin.
2024-07-28 14:08:32.759274: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 10, Chunks in use: 10. 5.0KiB allocated for chunks. 5.0KiB in use in bin. 5.0KiB client-requested in use in bin.
2024-07

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2] name: 

In [ ]:
train_images_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/synsets"
valid_images_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/valid_synsets"


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

train_generator = train_datagen.flow_from_directory(
    train_images_dir,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True,
    seed=1,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_images_dir,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True,
    seed=1,
    class_mode="categorical"
)

train_images_num = train_generator.samples
valid_images_num = valid_generator.samples

Found 3605 images belonging to 3 classes.
Found 264 images belonging to 3 classes.


In [ ]:
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%M%Y-%H%M%S")
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks_list = [tensorboard_callbacks]

In [ ]:
model.fit(
    train_generator,
    epochs=num_epochs,
    steps_per_epoch=train_images_num//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_images_num//batch_size,
    callbacks=callbacks_list,
    verbose=1
)

2024-07-28 14:03:27.142664: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/10


2024-07-28 14:03:30.822309: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 408.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-07-28 14:03:30.822368: W tensorflow/core/kernels/gpu_utils.cc:54] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2024-07-28 14:03:30.866627: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-07-28 14:03:31.124492: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-28 14:03:31.334880: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to al

ResourceExhaustedError: Graph execution error:

Detected at node vgg_net/conv2d_1/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/abhinav/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/abhinav/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/abhinav/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/abhinav/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/abhinav/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py